In [54]:
# example derived from /home/woody/caph/mpp228/ML/CTA/dl1-data-handler_stage1/notebooks/dl1reading_stage1.ipynb
# conda activate dl1dh_s1
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import time
import sys
sys.path.append("../../") # go to parent dir
import iact_dnn_utils as utils
import tables as tb

from dl1_data_handler.reader import DL1DataReaderSTAGE1, DL1DataReaderDL1DH

In [55]:
import importlib
importlib.reload(utils)

<module 'iact_dnn_utils' from '/home/woody/caph/mpp228/ML/iact_dnn/general/notebooks/../../iact_dnn_utils.py'>

In [56]:
cdict = {'mode':'mono',
         #'tel':'MST_MST_FlashCam',
         #'tel':'LST_LST_LSTCam',
        'Etrue_min':0.3,
         #'Etrue_min':None,
        'n_events':10**4,
         'n_events_tot':10**6,
         'folder_dl1':'/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00\
/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/',
         'folder_square':'/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00\
/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_square/',
         'event_type':'proton', # 'proton', 'gamma', 'electron',
         'mapping_settings': {'camera_types':['HESS-I']},
        }

In [57]:
test = False
if test:
    cdict['n_events'] = 1000
    cdict['n_events_tot'] = 2000
    
event_type = cdict['event_type'] # 'proton', 'gamma-diffuse', 'electron'
evt = event_type.split('-')[0]
mode = cdict['mode'] # 'mono', 'stereo'
if mode == 'mono':
    ntels = 1
else:
    ntels = 4 # generalize in the future?

mapping_settings = cdict['mapping_settings']

Etrue_min = cdict['Etrue_min']
if Etrue_min is not None:
    event_selection = [{'col_name': "true_energy", 'min_value':Etrue_min}]
else:
    event_selection = None

#n_events = 1000 #10**4
n_events = cdict['n_events'] # events per file
n_events_tot = cdict['n_events_tot'] # total number of events
print(n_events, mode)

#folder = '/home/saturn/caph/mpp228/CTA_data/Prod5_GRID/'
#folder_in = folder + 'Prod5_Paranal_AdvancedBaseline_NSB1x_%s_North_20deg_ctapipe_v0.10.5_DL1/' % event_type
folder_in = cdict['folder_dl1'].replace('EventPlaceholder', event_type)
fns = os.listdir(folder_in)
fns = [fn for fn in fns if fn.endswith('h5') and fn.startswith(event_type)]
print(len(fns), fns[0])

folder_out = cdict['folder_square']
if not os.path.isdir(folder_out):
    os.mkdir(folder_out)
    
fn_out = folder_out + utils.get_square_images_fn(cdict)

print('input folder:', folder_in)
print('output folder:', folder_out)

channels_list = ['image', 'image_mask', 'peak_time',]
tel_pars_list = ['obs_id', 'event_id', 'tel_id', 
                 'camera_frame_hillas_intensity', 'camera_frame_hillas_x', 'camera_frame_hillas_y', 
                 'camera_frame_hillas_width', 'camera_frame_hillas_length',
                 'camera_frame_hillas_psi', 'camera_frame_hillas_skewness', 'camera_frame_hillas_kurtosis', 
                 'camera_frame_hillas_r', 'camera_frame_hillas_phi',
            'morphology_num_pixels', 'morphology_num_medium_islands', 'morphology_num_large_islands',
            'leakage_pixels_width_1', 'leakage_pixels_width_2', 
            'leakage_intensity_width_1', 'leakage_intensity_width_2',
            'intensity_mean'] # tel parameters in cta-pipe .h5
if mode == 'stereo':
    tel_pars_list_extra = ['tel_trigger'] # tel parameters in cta-pipe .h5
else:
    tel_pars_list_extra = []
mc_info_list = ["true_energy", "true_alt", "true_az"] # mc parameters
groups = ['mc_pars', 'tel_pars', 'tel_pars_extra']

print(channels_list, tel_pars_list, mc_info_list)

def fn2reader(fn, mode='mono'):
    return DL1DataReaderSTAGE1([fn],
                             mode=mode,
                             #example_identifiers_file="./examples.h5",
                             #selected_telescope_types=selected_telescope_types,
                             mapping_settings=mapping_settings,
                             #selected_telescope_ids={"LST_LST_LSTCam": LSTcams},
                             #image_channels = ['image', 'peak_time', 'image_mask'],
                             image_channels=channels_list,
                             parameter_list=tel_pars_list,
                             event_info=mc_info_list,
                             event_selection=event_selection)

10000 mono
361 proton_20deg_180deg_run20526___phase2d2_desert-ws0-nsb1.00.h5
input folder: /home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/
output folder: /home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_square/
['image', 'image_mask', 'peak_time'] ['obs_id', 'event_id', 'tel_id', 'camera_frame_hillas_intensity', 'camera_frame_hillas_x', 'camera_frame_hillas_y', 'camera_frame_hillas_width', 'camera_frame_hillas_length', 'camera_frame_hillas_psi', 'camera_frame_hillas_skewness', 'camera_frame_hillas_kurtosis', 'camera_frame_hillas_r', 'camera_frame_hillas_phi', 'morphology_num_pixels', 'morphology_num_medium_islands', 'morphology_num_large_islands', 'leakage_pixels_width_1', 'leakage_pixels_width_2', 'leakage_intensity_width_1', 'leakage_intensity_width_2', 'intensity_mean'] ['true_energy', 'true_a

In [58]:
fns0 = os.listdir(folder_in)
fns = [fn for fn in fns0 if fn.endswith('h5') and fn.startswith(event_type)]
print(len(fns0))
print(len(fns))


921
361


In [59]:
out_dict = {key:{} for key in groups}
for key in mc_info_list:
    out_dict['mc_pars'][key] = np.zeros(n_events)
for key in tel_pars_list:
    out_dict['tel_pars'][key] = np.zeros((n_events, ntels))
for key in tel_pars_list_extra:
    out_dict['tel_pars_extra'][key] = np.zeros((n_events, ntels))

In [60]:
if test:
    fn = folder_in + fns[0]
    reader = fn2reader(fn, mode=mode)
    print(len(reader), mode)
    if mode == 'mono':
        print(reader[0][0].shape)
    else:
        print(reader[0][1].shape)


In [61]:
# there seems to be memory leakage in DL1DH: it cannot read more than about 500 files
# one needs to write the last filenumber for input and output here and restart the kernel
# the calculation will start with these files + 1
# for the initial run put i = -1, out_file_number = 0
if event_type == 'proton':
    i = 357
    out_file_number = 38
elif event_type == 'gamma':
    i = 330
    out_file_number = 51
elif event_type == 'electron':
    i = 194
    out_file_number = 30

nout = 0
nout_tot = 0
finish = False

In [62]:
%%time
t0 = time.time()
t_read = 0.

if mode == 'mono':
    i0 = 0
else:
    i0 = 1

i += 1
while nout_tot < n_events_tot or finish:
    fn = folder_in + fns[i]
    t_read0 = time.time()
    
    skip = False
    try:
        reader = fn2reader(fn, mode=mode)
    except tb.NoSuchNodeError:
        print('problem with file:')
        print(fn)
        print('skipping it\n')
        skip = True
        #break
        #finish = True
    except:
        print('problem with file:')
        print(fn)
        print(i)
        break
        #finish = True
        
    if not skip:
        t_read += time.time() - t_read0
        if 'image' not in out_dict:
            im_dims = reader[0][i0].shape[i0:i0+2]
            for key in channels_list:
                if mode == 'stereo':
                    out_dict[key] = np.zeros((n_events, im_dims[0], im_dims[1], ntels), dtype=np.float32)
                else:
                    out_dict[key] = np.zeros((n_events, im_dims[0], im_dims[1]), dtype=np.float32)
        for k in range(len(reader)):
            if nout + k == n_events:
                nout_tot += n_events
                nout = -k
                out_file_number += 1
                fn_out_running = fn_out.replace('.h5', '_file%i.h5' % out_file_number)
                if test:
                    fn_out_running = fn_out_running.replace('.h5', '_test.h5')
                if os.path.isfile(fn_out_running):
                    os.remove(fn_out_running)
                with h5py.File(fn_out_running, 'w') as h5_out:
                    for key in out_dict.keys():
                        if key in h5_out:
                            del h5_out[key]
                        if key in groups:
                            h5_out.create_group(key)
                            for vkey in out_dict[key].keys():
                                h5_out[key][vkey] = out_dict[key][vkey]
                        else:
                            h5_out[key] = out_dict[key]
                print('write %i events to file:' % n_events)
                print('i = %i' %i)
                print(fn_out_running)

            #out_dict['images'][nout + k] = reader[k][0][:,:,0]

            if mode == 'stereo':
                trigger = reader[k][0][:]
            if channels_list:
                images = reader[k][i0][:]
            if tel_pars_list:
                pars = reader[k][i0+1].T
            if mc_info_list:
                mc_pars = reader[k][i0+2:i0+2+len(mc_info_list)]

            for m, key in enumerate(channels_list):
                if mode == 'mono':
                    out_dict[key][nout + k] = images[:,:,m]
                else:
                    out_dict[key][nout + k] = np.transpose(images.T[m], (1,0,2))
            for m, key in enumerate(tel_pars_list):
                out_dict['tel_pars'][key][nout + k] = pars[m]
            for m, key in enumerate(mc_info_list):
                out_dict['mc_pars'][key][nout + k] = mc_pars[m]
            for key in tel_pars_list_extra:
                if key == 'tel_trigger' and mode == 'stereo':
                    out_dict['tel_pars_extra'][key][nout + k] = trigger
        nout += len(reader)
        del reader
    i += 1
    if i == len(fns):
        break
    if i % 10 == 0 or test:
        print('%i files, %i events, %.2f s to read, %.2f s past' % 
              (i, (nout_tot + nout), t_read, time.time() - t0))
    


300 files, 8733 events, 3.70 s to read, 34.19 s past
write 10000 events to file:
i = 301
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_square/proton_10000_images_mono_Etrue_min0.3TeV_file32.h5
310 files, 19162 events, 8.55 s to read, 77.06 s past
write 10000 events to file:
i = 310
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_square/proton_10000_images_mono_Etrue_min0.3TeV_file33.h5
320 files, 29743 events, 13.12 s to read, 119.87 s past
write 10000 events to file:
i = 320
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_square/proton_10000_images_mono_Etrue_min0.3TeV_file34.h5
write 10000 events to file:
i = 329
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton

In [63]:
nout_tot, nout, n_events_tot

(70000, 4075, 1000000)

In [64]:
folder_out

'/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_square/'

In [65]:
os.listdir(folder_out)

['proton_10000_images_mono_Etrue_min0.3TeV_file14.h5',
 'electron_10000_images_mono_Etrue_min0.3TeV_file13.h5',
 'proton_10000_images_mono_Etrue_min0.3TeV_file5.h5',
 'electron_10000_images_mono_Etrue_min0.3TeV_file9.h5',
 'electron_10000_images_mono_Etrue_min0.3TeV_file6.h5',
 'proton_10000_images_mono_Etrue_min0.3TeV_file2.h5',
 'electron_10000_images_mono_Etrue_min0.3TeV_file12.h5',
 'electron_10000_images_mono_Etrue_min0.3TeV_file16.h5',
 'proton_10000_images_mono_Etrue_min0.3TeV_file22.h5',
 'gamma_10000_images_mono_Etrue_min0.3TeV_file24.h5',
 'proton_10000_images_mono_Etrue_min0.3TeV_file16.h5',
 'electron_10000_images_mono_Etrue_min0.3TeV_file5.h5',
 'gamma_10000_images_mono_Etrue_min0.3TeV_file16.h5',
 'proton_10000_images_mono_Etrue_min0.3TeV_file9.h5',
 'electron_10000_images_mono_Etrue_min0.3TeV_file26.h5',
 'gamma_10000_images_mono_Etrue_min0.3TeV_file50.h5',
 'gamma_10000_images_mono_Etrue_min0.3TeV_file31.h5',
 'electron_10000_images_mono_Etrue_min0.3TeV_file1.h5',
 'gam

In [66]:
if 1:
    fn_test2 = folder_in + fns[i]
    print(fn_test2)
    reader = fn2reader(fn_test2, mode=mode)
    #del reader
    if 0:
        try:
            reader = fn2reader(fn_test2, mode=mode)
        except:
            print('skip')

IndexError: list index out of range

In [ ]:
len(fns)

In [ ]:
test_output = 1
if test_output:
    with h5py.File(fn_out_running, 'r') as h5_test:
        print(h5_test.keys())
        print(h5_test['tel_pars'].keys())
        print(h5_test['mc_pars'].keys())
        print(h5_test['image'].shape)
        tel_ids = np.array(h5_test['tel_pars']['tel_id'], dtype=int)


In [ ]:
#tel_ids = tel_ids[:,0]
plt.hist(tel_ids, bins=np.arange(4.5, 29.5, 1))

In [ ]:
if test:
    if 'image' in out_dict:
        plt.imshow(out_dict['image'][0,:,:,0])

In [ ]:
if test:
    with h5py.File(fn_out_running, 'r') as h5_test:
        plt.imshow(h5_test['image'][0,:,:,0])

In [ ]:
if test:
    with h5py.File(fn_out_running, 'r') as h5_test:
        plt.imshow(h5_test['image_mask'][0,:,:,0])

In [ ]:
if test:
    h5_test = h5py.File(fn_out_running, 'r')

In [ ]:
if test:
    for i in range(5):
        EE = h5_test['mc_pars']['true_energy'][i]
        hi = h5_test['tel_pars']['hillas_intensity'][i]
        hx = h5_test['tel_pars']['hillas_x'][i]
        print('%i, Energy %.3f' % (i, EE))
        print('hillas intensity %s' % hi)
        print('hillas x %s' % hx)


In [ ]:
if test:
    i = 9
    xi = 110 * (1 + h5_test['tel_pars']['hillas_x'][i]) / 2
    yi = 110 * (1 + h5_test['tel_pars']['hillas_y'][i]) / 2
    plt.plot(xi, yi, marker='s')

In [ ]:
if test:
    h5_test.close()

In [ ]:
if test:
    # remove test files
    for fn in os.listdir(folder_out):
        if fn.find('_test.h5') > 0:
            os.remove(folder_out + fn)